# Part 2: Tensor Products and Clebsch–Gordan Decomposition

**6.7970/8.750 Symmetry and its Application to Machine Learning**

Continuing with $D_4$ from Part 1, we explore how irreps combine under **tensor products**. By the end you will:

1. Understand what a tensor product representation is and how to compute it
2. Decompose tensor products back into irreps (the **Clebsch–Gordan problem**)
3. Build the full **irrep multiplication table** for $D_4$
4. Verify results using characters (the shortcut!)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/atomicarchitects/symm4ml-colabs/blob/main/decomposition_lecture/02_tensor_products.ipynb)

## Setup

In [ ]:
%%capture
!pip install https://symm4ml.mit.edu/_static/symm4ml_s26/symm4ml/symm4ml_latest.zip

In [20]:
import numpy as np
from symm4ml import groups, linalg, rep, vis
from symm4ml.utils import match_character_tables, print_character_table, print_multiplication_table

---
## 1. Recap: $D_4$ and its irreps

We rebuild $D_4$ and find its irreps in standard order (see Part 1 for details).

In [ ]:
# Build D4
C4 = np.array([[np.cos(np.pi/2), -np.sin(np.pi/2)],
               [np.sin(np.pi/2),  np.cos(np.pi/2)]])
sigma_v = np.array([[-1., 0.], [0., 1.]])
D4 = np.array(groups.generate_group(np.array([sigma_v, C4]))[::-1])
table_D4 = groups.make_multiplication_table(D4)
h = len(table_D4)

# Find and label irreps (see Part 1 for the matching procedure)
np.random.seed(42)
D4_irreps_raw = rep.infer_irreps(table_D4)

# Reorder to standard: A1, A2, B1, B2, E
conj_classes = groups.conjugacy_classes(table_D4)
conj_list = list(conj_classes)
char_table = rep.character_table(D4_irreps_raw, conj_list)

ref_chars = np.array([
    [1,  1,  1,  1,  1],   # A1
    [1,  1,  1, -1, -1],   # A2
    [1, -1,  1,  1, -1],   # B1
    [1, -1,  1, -1,  1],   # B2
    [2,  0, -2,  0,  0],   # E
])

row_perm, col_perm = match_character_tables(char_table, ref_chars)

names = ['A\u2081', 'A\u2082', 'B\u2081', 'B\u2082', 'E']
irreps = [D4_irreps_raw[i] for i in row_perm]
conj_std = [conj_list[j] for j in col_perm]

print("D\u2084 irreps in standard order:")
for name, ir in zip(names, irreps):
    print(f"  {name}: dim {ir.shape[1]}")


---
## 2. What is a tensor product representation?

Given two representations $\Gamma_i$ (dim $\ell_i$) and $\Gamma_j$ (dim $\ell_j$), their **tensor product** $\Gamma_i \otimes \Gamma_j$ is a new representation of dimension $\ell_i \cdot \ell_j$, defined by:

$$(\Gamma_i \otimes \Gamma_j)(g) = \Gamma_i(g) \otimes \Gamma_j(g)$$

where $\otimes$ on the right is the **Kronecker product** of matrices.

Let's start with a simple example: $A_2 \otimes E$.

In [ ]:
A2 = irreps[1]  # dim 1
E  = irreps[4]  # dim 2

A2_x_E = rep.tensor_product(A2, E)
print(f"A\u2082 shape: {A2.shape}  (dim {A2.shape[1]})")
print(f"E  shape: {E.shape}  (dim {E.shape[1]})")
print(f"A\u2082 \u2297 E shape: {A2_x_E.shape}  (dim {A2_x_E.shape[1]})")

print(f"\nIs A\u2082 \u2297 E a valid representation? {rep.is_a_representation(table_D4, A2_x_E)}")
print(f"Is it irreducible?  {rep.is_an_irrep(table_D4, A2_x_E)}")

$A_2 \otimes E$ is 2-dimensional and irreducible — so it must be isomorphic to $E$ itself (the only 2D irrep of $D_4$). Let's check:

In [ ]:
print(f"A\u2082 \u2297 E \u2245 E?  {rep.are_isomorphic(A2_x_E, E)}")

This makes sense: multiplying by the 1D sign representation just flips some signs but doesn't change the dimension or structure of $E$.

---
## 3. A more interesting example: $E \otimes E$

The tensor product of the 2D irrep with itself gives a **4-dimensional** representation. This one *cannot* be irreducible (the largest irrep of $D_4$ is 2D), so it must decompose into smaller pieces.

In [ ]:
E_x_E = rep.tensor_product(E, E)
print(f"E \u2297 E shape: {E_x_E.shape}  (dim {E_x_E.shape[1]})")
print(f"Is it irreducible?  {rep.is_an_irrep(table_D4, E_x_E)}")
print(f"\nLet's look at the matrices:")
vis.plot_matrices(E_x_E, figsize=(12, 3));

### Decomposing $E \otimes E$

We use `rep.decompose_rep_into_irreps` to find the irreducible pieces, then identify each one.

In [ ]:
sub_irreps = rep.decompose_rep_into_irreps(E_x_E)

print(f"E \u2297 E decomposes into {len(sub_irreps)} irreps:\n")
for k, sub in enumerate(sub_irreps):
    # Identify which standard irrep this matches
    label = '?'
    for name, ir in zip(names, irreps):
        if rep.are_isomorphic(sub, ir):
            label = name
            break
    print(f"  Component {k+1}: dim {sub.shape[1]}  \u2192  {label}")

dims = [s.shape[1] for s in sub_irreps]
print(f"\nDimension check: {' + '.join(map(str, dims))} = {sum(dims)} = dim(E\u2297E) = {E_x_E.shape[1]}  \u2713")

So $E \otimes E = A_1 \oplus A_2 \oplus B_1 \oplus B_2$. All four 1D irreps appear!

This is a key result: the tensor product of the 2D irrep with itself contains every 1D irrep exactly once.

---
## 4. The character shortcut

Computing tensor products and decomposing them matrix-by-matrix works, but there's a much faster way using **characters**:

$$\chi_{\Gamma_i \otimes \Gamma_j}(g) = \chi_{\Gamma_i}(g) \cdot \chi_{\Gamma_j}(g)$$

Characters of a tensor product are just the **product of characters**. And to decompose, we use the **orthogonality formula**:

$$n_k = \frac{1}{h} \sum_g \chi_{\Gamma_k}^*(g) \, \chi_{\text{product}}(g)$$

Let's verify this for $E \otimes E$:

In [ ]:
# Character table in standard order
chi = rep.character_table(irreps, conj_list)
class_sizes = np.array([len(cc) for cc in conj_list])

# Characters of E (last row)
chi_E = chi[4]  # E

# Characters of E⊗E = product of characters
chi_ExE = chi_E * chi_E
print(f"\u03c7(E):     {np.round(chi_E.real, 1)}")
print(f"\u03c7(E\u2297E):  {np.round(chi_ExE.real, 1)}")

# Decompose using orthogonality
print(f"\nMultiplicity of each irrep in E\u2297E:")
for i, name in enumerate(names):
    n_i = np.sum(class_sizes * np.conj(chi[i]) * chi_ExE).real / h
    print(f"  n({name}) = {n_i:.1f}")

The character method gives the same answer instantly, without ever computing the actual decomposition matrices!

---
## 5. The full irrep multiplication table

Now let's compute **all** tensor products $\Gamma_i \otimes \Gamma_j$ and build a complete multiplication table. We'll use both methods to cross-check.

In [ ]:
# Compute the full irrep multiplication table using the course library
dp_table = rep.direct_product_table(irreps, names)

# Format for printing: convert sets to ⊕-joined strings
mul_table = [['⊕'.join(sorted(dp_table[i, j])) for j in range(len(names))]
             for i in range(len(names))]

print_multiplication_table(mul_table, names, title=f"Irrep multiplication table for D₄")

### Reading the table

Key patterns to notice:
- **$A_1$ is the identity**: $A_1 \otimes \Gamma = \Gamma$ for all $\Gamma$
- **1D $\times$ 1D = 1D**: Tensor products of 1D irreps give 1D irreps
- **1D $\times$ 2D = 2D**: $A_2 \otimes E = E$, etc.
- **$E \otimes E$ is special**: It decomposes into all four 1D irreps

---
## 6. Same table from characters alone

The character shortcut lets us build the entire table without ever constructing matrices. This is how it's done in practice for larger groups.

In [ ]:
# Build the same table using only characters (no matrix decomposition needed!)
mul_table_chi = []
for i in range(len(names)):
    row = []
    for j in range(len(names)):
        chi_prod = chi[i] * chi[j]
        labels = []
        for k in range(len(names)):
            n_k = int(round(np.sum(class_sizes * np.conj(chi[k]) * chi_prod).real / h))
            labels.extend([names[k]] * n_k)
        row.append('⊕'.join(sorted(labels)))
    mul_table_chi.append(row)

print_multiplication_table(mul_table_chi, names, title="Multiplication table via characters")
print("\n(Should match the table above exactly!)")

---
## 7. Visualizing the block-diagonal structure

Let's see what $E \otimes E$ looks like before and after decomposition.

In [ ]:
print("E \u2297 E (scrambled):")
vis.plot_matrices(E_x_E, figsize=(12, 3));

# Reconstruct as a direct sum of the identified irreps
sub_irreps_ExE = rep.decompose_rep_into_irreps(E_x_E)
direct = rep.direct_sum_multiple(sub_irreps_ExE)
print("\nE \u2297 E (block-diagonalized):")
vis.plot_matrices(direct, figsize=(12, 3));

After decomposition, we see four 1×1 blocks along the diagonal — the four 1D irreps $A_1, A_2, B_1, B_2$.

---
## Summary

| Concept | What we learned |
|---|---|
| **Tensor product** | $\Gamma_i \otimes \Gamma_j$ is a new rep of dim $\ell_i \cdot \ell_j$ |
| **Clebsch–Gordan decomposition** | Any tensor product decomposes into a direct sum of irreps |
| **Character shortcut** | $\chi(\Gamma_i \otimes \Gamma_j) = \chi(\Gamma_i) \cdot \chi(\Gamma_j)$, then use orthogonality |
| **Multiplication table** | Captures all possible ways irreps combine — fundamental to building equivariant models |

**Next up** (Part 3): We apply these tools to a larger group ($T_d$) to see how the algorithms scale.